<h2>Classification</h2>

Authors: Casper Smit and Samantha Visbeek

In [5]:
# imports
import pandas as pd
import numpy as np
from import_data import *
from util import *


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/caspeerrr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# create dataframe
df = import_merge()
df = clean_dataframe(df)

# drop rows where hdi_score is Nan
df = df[df['hdi_score'].notna()]
df = df.reset_index(drop=True)
df.head()

  0%|                                                                                             | 0/8 [00:00…

Country Cleaning Report:
	933 values cleaned (11.0%)
	37 values unable to be parsed (0.44%), set to NaN
Result contains 8444 (99.56%) values in the correct format and 37 null values (0.44%)


,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,...,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS),Developed / Developing Countries,Session,Year,Speech,hdi_score,hdi_class,country_cleaned
0,1.0,World,19.0,Americas,419.0,Latin America and the Caribbean,5.0,South America,Guyana,328.0,...,NaN,NaN,x,Developing,45,1990,﻿There is now abroad in the world a mood of op...,0.548,low,Guyana
1,1.0,World,142.0,Asia,35.0,South-eastern Asia,NaN,NaN,Indonesia,360.0,...,NaN,NaN,NaN,Developing,45,1990,"Mr. President, it gives me great pleasure to o...",0.523,low,Indonesia
2,1.0,World,2.0,Africa,202.0,Sub-Saharan Africa,14.0,Eastern Africa,Rwanda,646.0,...,x,x,NaN,Developing,45,1990,﻿It is a great honour for me and for the entir...,0.248,low,Rwanda
3,1.0,World,150.0,Europe,151.0,Eastern Europe,NaN,NaN,Hungary,348.0,...,NaN,NaN,NaN,Developed,45,1990,﻿May I first of all extend to Mr. Guido de Mar...,0.708,high,Hungary
4,1.0,World,2.0,Africa,202.0,Sub-Saharan Africa,11.0,Western Africa,Senegal,686.0,...,x,NaN,NaN,Developing,45,1990,﻿The delegation of Senegal warmly congratulate...,0.376,low,Senegal


<h3>Text Processing</h3>


In [7]:
# remove all names from the speeches
df['SpeechNoNames'] = df['Speech'].apply(remove_names)

# text processing
df['Speech'] = df['Speech'].apply(clean_string).apply(remove_stopw)
df['SpeechNoNames'] = df['SpeechNoNames'].apply(clean_string).apply(remove_stopw)

In [8]:
# remove all words that only occur once in the whole dataframe
df['Speech'] = remove_single_occurence(df['Speech'])
df['SpeechNoNames'] = remove_single_occurence(df['SpeechNoNames'])

In [9]:
# transform speeches list to string
df['Speech'] = df['Speech'].apply(lambda x: ' '.join(x))
df['SpeechNoNames'] = df['SpeechNoNames'].apply(lambda x: ' '.join(x))

In [10]:
# make dataframes of the tf-idf of the words in each speech
df_tfidf = tf_idf(df['Speech'])
df_tfidfNoNames = tf_idf(df['SpeechNoNames'])

/home/caspeerrr/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
# get the features and the target for the classification
features = df_tfidf
featuresNoNames = df_tfidfNoNames

target = df['hdi_class']

In [12]:
# create the logistic regression model and predict
pred, classes, weights, accuracy, precision, recall = logreg(features, target)
predNN, classesNN, weightsNN, accuracyNN, precisionNN, recallNN = logreg(featuresNoNames, target)

In [13]:
print('Accuracy logistic regression model: ', accuracy)
print('Accuracy logistic regression model without names: ', accuracyNN)
print('\n')

print('Precision logistic regression model: ', precision)
print('Precision logistic regression model without names: ', precisionNN)
print('\n')

print('Recall logistic regression model: ', recall)
print('Recall logistic regression model without names: ', recallNN)

Accuracy logistic regression model:  0.7969849246231155
Accuracy logistic regression model without names:  0.7065326633165829


Precision logistic regression model:  [0.67755102 0.88086643 0.72427984 0.9       ]
Precision logistic regression model without names:  [0.59174312 0.76369863 0.62151394 0.83333333]


Recall logistic regression model:  [0.73451327 0.91729323 0.69565217 0.828     ]
Recall logistic regression model without names:  [0.57079646 0.83834586 0.61660079 0.78      ]


In [14]:
n = 10

# calcualte the top n most distincive words for each class
class_words_dict = top_n_distinctive_words(n, df_tfidf, classes, weights)
class_words_dictNN = top_n_distinctive_words(n, df_tfidfNoNames, classesNN, weightsNN)

In [15]:
# print the top n most distinctive words for classification
print('With names:')
display_words(class_words_dict)
print('Without names:')
display_words(class_words_dictNN)

With names:
high
serb, sails, bulbs, domingo, mexicans, algebra, lebanese, sexually, kutesas, carib


low
mad, sucre, delegating, pakhtunkhwa, paperwork, guilty, yeltsins, africa, afresh, malaria


medium
philippine, bothers, micronationalism, dominica, guy, egregious, namgyel, equatoria, kyrgyz, fw


very high
iceberg, rightness, spade, qarase, zeal, kordofan, humanistic, janus, bahama, europe


Without names:
high
stature, regimes, revolts, ethics, occupations, moderating, aggressed, slurs, region, islamic


low
divisiveness, tributary, situated, subprime, partnering, election, nation, pays, delegating, contiguous


medium
climactic, democracies, islamic, millennial, soccer, developing, penury, democrat, graduating, drudgery


very high
strongholds, yearns, ahistorical, acting, necks, rightness, responsibilities, humanistic, valued, migrating




In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# get the features and the target for the classification
features = df_tfidf
featuresNoNames = df_tfidfNoNames

target = df['Year']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
model = LinearRegression().fit(X_train, y_train)
pred = model.predict(X_test)
print(mean_absolute_error(pred, X_test))


In [1]:
df

NameError: name 'df' is not defined